In [25]:
import pandas as pd
import spacy
from py2neo import Graph, Node, Relationship
import os

In [36]:
df = pd.read_pickle('data.pkl')
df['Case ID'] = df.index
df.head()

,Title,Summary,Full Text,Key Issues,Relevant Statutes and Provisions,Precedents Cited,Legal Doctrines,Jurisdiction,Date,Case ID
0,Gujarat Jhm Hotels Ltd vs Rajasthali Resorts A...,## 1. Case Title\n\n* **Case Name:** Gujarat J...,Delhi High Court\nGujarat Jhm Hotels Ltd vs Ra...,The case raised a key issue about the correct ...,"**\n* Arbitration and Conciliation Act, 1996 (...",**\n* State of W.B. v. Associated Contractors ...,**\n* **Party Autonomy:** The right of parties...,Delhi High Court,17 January 2023,0
1,Span Healthcare Pvt Ltd vs Vishal Sharma,## 1. Case Title\n\n* **Case Name:** Span Hea...,Delhi High Court - Orders\nSpan Healthcare Pvt...,Maintainability of the Revision Petition filed...,**\n* Section 115 of the Code of Civil Procedu...,**\n* Shah Babulal Khimji vs. Jayaben D. Kania...,**\n* Interpretation of Statutes - The court a...,Delhi High Court,16 April 2021,1
2,Gujarat Energy Transmission Corporation Ltd GE...,## Gujarat Energy Transmission Corporation Lt...,Gujarat High Court\nGujarat Energy Transmissio...,The primary legal issue was whether Section 19...,"**\n* Arbitration and Conciliation Act, 1996 (...",**\n* JMC Projects (India) Ltd. vs. Mechtech E...,**\n* Statutory Interpretation: The Court had ...,Gujarat High Court,10th April 2017,2
3,India Oil Corporation Ltd And Another vs The C...,## 1. Case Title\n\n* **Case Name:** India Oil...,Allahabad High Court\nIndia Oil Corporation Lt...,Whether the arbitrator had jurisdiction to con...,"**\n* Indian Arbitration Act, 1940 (Old Act, 1...",**\n* Thyssen Stahlunion GMBH etc. Vs. Steel A...,**\n* Res judicata (a matter that has been dec...,Allahabad High Court,6th September 2023,3
4,Mr A Krishna Murthy vs Mr Shantharaj,## Case Title \n\n* **Case Name:** Mr. A. Kri...,Karnataka High Court\nMr. A. Krishna Murthy vs...,Whether the dispute between the parties consti...,"**\n* **Commercial Courts Act, 2015:** Particu...",**\n* **Ambalal Sarabhai Enterprises Limited v...,**\n* **Purposive Interpretation:** The court ...,Karnataka High Court,10th January 2024,4


In [37]:
nlp = spacy.load("en_core_web_sm")

# Process a case summary
doc = nlp(df['Summary'][0])
# for ent in doc.ents:
#     print(ent.text, ent.label_)

In [71]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "lakshya@"))
for index, row in df.iterrows():
    # Start a new transaction for each row
    tx = graph.begin()

    try:
        # Create Case node
        case_node = Node("Case", case_id=row["Case ID"], title=row["Title"], summary=row["Summary"], date=row["Date"])
        tx.create(case_node)

        # Create Statute node if it doesn't exist already
        if row["Relevant Statutes and Provisions"]:
            statute_name = row["Relevant Statutes and Provisions"]
            statute_node = graph.nodes.match("Statute", name=statute_name).first()  # Check if the statute exists
            if statute_node is None:
                statute_node = Node("Statute", name=statute_name)
                tx.create(statute_node)
            relationship = Relationship(case_node, "PERTAINS_TO", statute_node)
            tx.create(relationship)

        # Create Precedent node if it doesn't exist already
        if row["Precedents Cited"]:
            precedent_name = row["Precedents Cited"]
            precedent_node = graph.nodes.match("Precedent", name=precedent_name).first()  # Check if the precedent exists
            if precedent_node is None:
                precedent_node = Node("Precedent", name=precedent_name)
                tx.create(precedent_node)
            relationship = Relationship(case_node, "CITES", precedent_node)
            tx.create(relationship)

        # Create Legal Doctrine node if it doesn't exist already
        if row["Legal Doctrines"]:
            doctrine_name = row["Legal Doctrines"]
            doctrine_node = graph.nodes.match("Doctrine", name=doctrine_name).first()  # Check if the doctrine exists
            if doctrine_node is None:
                doctrine_node = Node("Doctrine", name=doctrine_name)
                tx.create(doctrine_node)
            relationship = Relationship(case_node, "FALLS_UNDER", doctrine_node)
            tx.create(relationship)

        # Create Jurisdiction node if it doesn't exist already
        if row["Jurisdiction"]:
            jurisdiction_name = row["Jurisdiction"]
            jurisdiction_node = graph.nodes.match("Jurisdiction", name=jurisdiction_name).first()  # Check if the jurisdiction exists
            if jurisdiction_node is None:
                jurisdiction_node = Node("Jurisdiction", name=jurisdiction_name)
                tx.create(jurisdiction_node)
            relationship = Relationship(case_node, "WITHIN", jurisdiction_node)
            tx.create(relationship)

        # Commit the transaction after processing the row
        graph.commit(tx)

    except Exception as e:
        print(f"Error occurred on row {index}: {e}")
        tx.rollback()  # Roll back the transaction if an error occurs